# UNCLASSIFIED

Transcribed from FOIA Doc ID: 6689695

https://archive.org/details/comp3321

(U) Most command line programs accept options and arguments, and many even provide help messages that indicate what options are available, and how they are to be used. For example, the utility program `mv` takes two arguments, and most often moves the first argument (the source) to the second (the destination). It has other ways of operating, which are enabled by optional flags and arguments; from a command prompt, type `mv --help` to see more. 

(U) There are several ways to enable this type of functionality in a Python program, and the best way to do it has been a source of contention. In particular, this lesson will cover the **argparse** module, which was added to the standard library in Python 2.7, and not the **optparse** module which was deprecated at that time. 

(U) Everything passed as arguments to a Python program is available in the interpreter as the list of strings in `sys.argv`. In an interactive session, `sys.argv` always starts out as `['']`. When running a script, `sys.argv[0]` is the name of the script. We start by examining what `sys.argv` looks like. Put the following commands in a file called `argtest.py` or similar: 

```python
import sys 
print (sys.argv) 
```

Running the following code cell will create this file for you:

In [ ]:
contents = '''import sys
print(sys.argv)
'''

with open('argtest.py', 'w') as f:
    f.write(contents)

(U) Close the file and execute it from the command line with some arguments: 

In [ ]:
# the '!' at the beginning tells jupyter to send what follows to the command line 
!python3 argtest.py -xzf --v foo --othervar=bar file1 file2

```
['argtest.py', '-xzf', '--v', 'foo', '--othervar=bar', 'file1', 'file2']
```

(U) In all of the argument parsing that follows, `sys.argv` will be involved, although that may happen either implicitly or explicitly. Although it is often unwise to do so within a script, `sys.argv` can be modified, for instance during testing within an interactive session, 

(U) Note that in Jupyter you still have `argv`, but it may not be what you expect. If you look at it, you'll see how this Python 3 kernel is being called: 

In [ ]:
import sys
print(sys.argv)

## (U) The Hard Way: `getopt` 

(U) For programs with only simple arguments, the **getopt** module provides functionality similar to the `getopt` function in C. The main method in the module is `getopt`, which takes a list of strings, usually `sys.argv[1:]` and parses it according to a string of options, with optional _long options_, which are allowed to have more than one letter; explanations are best left to examples. This method returns a pair of lists, one containing `(option, 
value)` tuples, the other containing additional positional arguments. These values must then be further processed within the program; it might be useful, for instance, to put the `(option, value)` tuples into a `dict`. If `getopt` receives an unexpected option, it throws an error. If it does not receive all the arguments it requests, no error is thrown, and the missing arguments are not present in the returned value. 

In [ ]:
import getopt 

In [ ]:
getopt.getopt('-a arg'.split(), 'a:' ) # a expects an argument

In [ ]:
getopt.getopt('-a arg'.split(), 'a:b') # no b, no problem

In [ ]:
getopt.getopt('-b arg -a my-file.txt'.split(), 'ab:') # my-file.txt is argument, not option

In [ ]:
getopt.getopt('-a arg --output=other-file.txt my-file.txt'.split(), 'a:b', ['output=']) # Long options

(U) For programs that use `getopt`, usage help must be provided manually, 

In [ ]:
def usage(): 
    print ("""usage: my_program.py -[abh] file1, file2, ...""") 

In [ ]:
# this won't actually find anything in Jupyter, since ipython3 probably doesn't have these options 
opts, args = getopt.getopt(sys.argv[1:], 'abh')

In [ ]:
opt_dict = dict(opts)

In [ ]:
if '-h' in opt_dict: 
    usage()

## (U) The `argparse` Module

(U) Integrated help is one of the benefits of **argparse**, along with the ability to specify both short and long options versions of arguments. There is some additional complication in setting up **argparse**, but it is the right thing to do for all but the most simple programs.

### (U) Basic Usage 

(U) The main class in **argparse** is `ArgumentParser`. After an `ArgumentParser` is instantiated, arguments are added to it with the `add_argument` method. After all the arguments are added, the `parse_args` method is called. By default, it reads from `sys.argv[1:]`, but can also be passed a list of strings, primarily for testing. Both positional arguments (required) and optional arguments indicated by flags are supported. An example will illustrate the operation.

In [ ]:
import argparse

In [ ]:
parser = argparse.ArgumentParser()

In [ ]:
parser.add_argument('n')

In [ ]:
parser.add_argument('-f')

In [ ]:
parser.add_argument('-i','--input')

In [ ]:
parser.print_help()

In [ ]:
parser.parse_args('abc -f xyz'.split())

In [ ]:
parser.parse_args('-f xyz abc --input=myfile.txt'.split())

In [ ]:
parser.parse_known_args('-f xyz abc --input=myfile.txt -o otherfile.txt'.split())

In [ ]:
args = parser.parse_args('-f xyz abc --input=myfile.txt'.split())

In [ ]:
args.f

(U) As seen in the final two lines, positional arguments and optioned arguments can come in any order, which is not the case with **getopt**. If multiple positional arguments are specified, they are parsed in the order in which they were added to the `ArgumentParser`. The object returned by `parse_args` is called a `Namespace`, but it is just an object which contains all the parsed data. Unless otherwise specified, the attribute names are derived from the option names. Positional arguments are used directly, while short and long flags have leading hyphens stripped and internal hyphens converted to underscores. If more than one flag is specified for an argument, the first long flag is used if present; otherwise, the first short flag is used. 

(U) Here is how argparse could look in your code.

```python
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('n')
parser.add_argument('-f')
parser.add_argument('-i', '--input')
args = parser.parse_args()

print(args)```

In [ ]:
contents = '''import argparse
parser = argparse.ArgumentParser()
parser.add_argument('n')
parser.add_argument('-f')
parser.add_argument('-i', '--input')
args = parser.parse_args()

print(args)
'''

with open('argparsetest.py', 'w') as f:
    f.write(contents)

(U) Now we can simulate running it: 

In [ ]:
!python3 argparsetest.py -f xyz abc --input=myfile.txt

In [ ]:
!python3 argparsetest.py -h 

### (U) Advanced Options 

(U) The `add_argument` method supports a large number of keyword arguments that configure behavior more finely than the defaults. For instance, the `type` argument will make the parser attempt to convert arguments to the specified type, such as `int`, `float`, or `file`. In fact, you could use any class for the type, as long as it can be constructed from a single string argument.

In [ ]:
parser = argparse.ArgumentParser()

In [ ]:
parser.add_argument('n', type=int)

In [ ]:
parser.parse_args('5'.split())

(U) The `nargs` keyword lets an argument specify a fixed or variable number of arguments to consume, which are then stored into a list. This applies to both positional and optional arugments. Giving `nargs` the value `'?'` makes positional arguments optional, in which case the `default` keyword is required. 

In [ ]:
parser = argparse.ArgumentParser()

In [ ]:
parser.add_argument('n', nargs=2)

In [ ]:
parser.add_argument('-m', nargs='*') # arbitrary arguments

In [ ]:
parser.parse_args('n1 n2 -m a b c'.split())

In [ ]:
parser.add_argument('o', nargs='?', default='OoO')

In [ ]:
parser.parse_args('n1 n2 oOo -m a b c'.split())

(U) The `default` keyword can also be used with optional arguments. When an optional argument is always used as a flag without parameters, it is also possible to use the `action='store_const'` and `const` keywords. In this case, when the option is detected, the `Namespace` is given an appropriately-named attribute with `const` as its value. If the option is not present in the parsed args, the attribute is created with the value given in `default`, or `None` if `default` isn't set.

In [ ]:
parser = argparse.ArgumentParser()

In [ ]:
parser.add_argument('-n', action='store_const', const=7)

In [ ]:
parser.add_argument('-b', action='store_true')

In [ ]:
parser.add_argument('-c', action='store_const', const=5, default=3)

In [ ]:
parser.parse_args([])

In [ ]:
parser.parse_args('-n -b'.split())

In [ ]:
parser.parse_args('-cbn'.split())

(U)The `action` keyword can take other arguments; for instance, `action='store_true'` and `action='store_false'` can be used instead of setting `const` to a boolean value. 

(U) Once again, we have only scratched the surface of a module, **argparse** in this case. Check out the [documentation](https://docs.python.org/3/library/argparse.html) for more details (e.g. changing attribute names with the `dest` keyword, writing custom action functions, providing different parsers for subprograms).

# UNCLASSIFIED

Transcribed from FOIA Doc ID: 6689695

https://archive.org/details/comp3321